In [48]:
import pandas as pd
#from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import Sequential
from keras.layers import Dense, Dropout
import keras.metrics
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from MLscores import calc_metrics, metrics_dict, cmvals, recall, hybridrecall
import keras.backend as K
import tensorflow as tf
from functools import partial
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, make_scorer
from xgboost import XGBClassifier
from check_and_prepare_dataset import load_dataset, prepare_dataset
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus
#import eli5
#from eli5.sklearn import PermutationImportance

In [56]:
#params={'ES_mindelta': 0.001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': 0.1, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 70}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params={'ES_mindelta': 0.0001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': 0.3, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 200.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params = {}
params['nn_auc']= {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h1'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh5'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1100.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}

In [64]:
models={}
for paramset in params:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop'])
    creatennmodel = partial(create_NN_model, params[paramset], X)
    models[paramset]={}
    models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
    models[paramset]['params'] = params[paramset]

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'pop', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_corine_gr2

/tmp/ipykernel_3222130/3430584326.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_corine_gr21', 'bin_corine_gr22']
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_samp

/tmp/ipykernel_3222130/3430584326.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'pop', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_corine_gr21', 'bin_corine_gr22']
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_n

/tmp/ipykernel_3222130/3430584326.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_36', 'bin_dir_max_7', 'bin_month_5', 'bin_corine_30', 'bin_dir_max_3', 'bin_corine_23', 'bin_weekday_4', 'bin_corine_35', 'bin_corine_11', 'bin_corine_3', 'bin_corine_10', 'bin_weekday_2', 'bin_dom_dir_7', 'bin_corine_25', 'bin_dir_max_4', 'bin_corine_5', 'bin_corine_33', 'bin_corine_17', 'bin_corine_31', 'bin_corine_21', 'bin_corine_18', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_24', 'bin_corine_41', 'bin_corine_29', 'bin_corine_16', 'bin_corine_20', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_corine_8', 'bin_dom_dir_4', 'bin_corine_28', 'bin_dom_dir_3', 'bin_corine_4', 'bin_month_9', 'bin_corine_9', 'bin_dom_dir_5', 'bin_corine_40', 'bin_corine_7', 'bin_month_8', 'bin_corine_6', 'bin_month_4', 

/tmp/ipykernel_3222130/3430584326.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_36', 'bin_dir_max_7', 'bin_month_5', 'bin_corine_30', 'bin_dir_max_3', 'bin_corine_23', 'bin_weekday_4', 'bin_corine_35', 'bin_corine_11', 'bin_corine_3', 'bin_corine_10', 'bin_weekday_2', 'bin_dom_dir_7', 'bin_corine_25', 'bin_dir_max_4', 'bin_corine_5', 'bin_corine_33', 'bin_corine_17', 'bin_corine_31', 'bin_corine_21', 'bin_corine_18', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_24', 'bin_corine_41', 'bin_corine_29', 'bin_corine_16', 'bin_corine_20', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_corine_8', 'bin_dom_dir_4', 'bin_corine_28', 'bin_dom_dir_3', 'bin_corine_4', 'bin_month_9', 'bin_corine_9', 'bin_dom_dir_5', 'bin_corine_40', 'bin_corine_7', 'bin_month_8', 'bin_corine_6', 'bin_month_4', 

/tmp/ipykernel_3222130/3430584326.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


In [65]:
paramssk={}
#RF best
paramssk['RF']={'algo': 'RF','bootstrap': True, 'class_weights': {0: 1, 1: 10}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 1000, 'max_features': 2.0, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 350}
#ET best
paramssk['ET']={'algo': 'XT', 'bootstrap': True, 'class_weights': {0: 1, 1: 1}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': None, 'max_features': 2.0, 'min_samples_leaf': 50, 'min_samples_split': 150, 'n_estimators': 800}
#XG
paramssk['XGB']={'algo': 'XGB', 'alpha': 60, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'gamma': 0.1, 'lambda': 21.0, 'max_depth': 44.0, 'n_estimators': 640, 'scale_pos_weight': 15, 'subsample': 0.6}
for skalgo in paramssk:
    print(paramssk[skalgo]['algo'])
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop'])
    models[skalgo]={}
    models[skalgo]['model'] = create_sklearn_model(paramssk[skalgo],X)
    models[skalgo]['params']=paramssk[skalgo]


RF
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_36', 'bin_dir_max_7', 'bin_month_5', 'bin_corine_30', 'bin_dir_max_3', 'bin_corine_23', 'bin_weekday_4', 'bin_corine_35', 'bin_corine_11', 'bin_corine_3', 'bin_corine_10', 'bin_weekday_2', 'bin_dom_dir_7', 'bin_corine_25', 'bin_dir_max_4', 'bin_corine_5', 'bin_corine_33', 'bin_corine_17', 'bin_corine_31', 'bin_corine_21', 'bin_corine_18', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_24', 'bin_corine_41', 'bin_corine_29', 'bin_corine_16', 'bin_corine_20', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_corine_8', 'bin_dom_dir_4', 'bin_corine_28', 'bin_dom_dir_3', 'bin_corine_4', 'bin_month_9', 'bin_corine_9', 'bin_dom_d

In [66]:
models

{'nn_auc': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f2b05119f30>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 5},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'pop',
    'corine_gr1',
    'corine_gr2',
    'corine_gr3',
    'corine_gr4',
    'corine_gr5',
    'corine_gr6',
    'corine_gr7',
    'corine_gr8',
    'corine_gr9'),
   'max_epochs': 2000,
   'metric': 'accuracy',
   'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}),
   'optimizer': {'adam_params': None, 'name': 'Adam'}}},
 'nn_h1': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f2a105a44c0>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 2},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'corine_gr1',
   

In [67]:
#models1={'RF':models['RF'], 'ET':models['ET']}
#allowgrowthgpus()
for modname in models:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         models[modname]['params']['feature_drop'])
    print(modname)
    models[modname]['model'].fit(X,y)
    break

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'pop', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_corine_gr2

In [70]:
results_lists={}
for modname in models:
    print(modname)
    #models[modname].fit(X,y)
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                             models[modname]['params']['feature_drop'])
    r=permutation_importance(models[modname], X, y, n_repeats=3,random_state=0, scoring='roc_auc')
    results_lists[modname]=[]
    for i in r.importances_mean.argsort()[::-1]:
        #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        if not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month']):
            results_dict={}
            results_dict['rank '+modname]=X.columns[i]
            results_dict['perm. imp.'+modname]=r.importances_mean[i]
            results_lists[modname]+=[results_dict]

nn_auc
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'pop', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_cor

TypeError: estimator should be an estimator implementing 'fit' method, {'model': <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2b05119f30>, 'params': {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}} was passed

In [40]:
X.columns

Index(['bin_corine_14', 'bin_corine_gr1', 'bin_corine_44', 'bin_corine_7',
       'bin_corine_18', 'bin_corine_41', 'bin_corine_gr32', 'max_temp',
       'dom_vel', 'bin_corine_5',
       ...
       'bin_month_8', 'bin_month_9', 'bin_month_10', 'bin_weekday_1',
       'bin_weekday_2', 'bin_weekday_3', 'bin_weekday_4', 'bin_weekday_5',
       'bin_weekday_6', 'bin_weekday_7'],
      dtype='object', length=105)

In [21]:
recall_score(y,ypred)

0.9983417447728912

In [72]:
rec1_scorer = make_scorer(recall_score, pos_label=1)
rec0_scorer = make_scorer(recall_score, pos_label=0)
results_lists={}
scoring={'recall1':rec1_scorer, 'recall0':rec0_scorer}
for modname in models:
    print(modname)
    #models[modname].fit(X,y)
    #ypred=models['nn_auc'].predict(X)
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                          models[modname]['params']['feature_drop'])
    r=permutation_importance(models[modname]['model'], X, y, n_repeats=1, scoring=scoring)
    for s in scoring:
        results_lists['%s %s'%(modname,s)]=[]
        for i in r[s].importances_mean.argsort()[::-1]:
            #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            if not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month']):
                results_dict={}
                results_dict['rank %s %s'%(modname,s)]=X.columns[i]
                results_dict['perm. imp. %s %s'%(modname,s)]=r[s].importances_mean[i]
                results_lists['%s %s'%(modname,s)]+=[results_dict]
    break

nn_auc
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_corine_gr31', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_corine_gr24', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_corine_gr23', 'bin_month_8', 'bin_month_4', 'bin_corine_gr1', 'bin_dir_max_1', 'bin_corine_gr4', 'bin_month_3', 'bin_dom_dir_1', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'pop', 'bin_weekday_3', 'bin_corine_gr33', 'bin_corine_gr5', 'bin_dir_max_5', 'bin_cor

In [14]:
#r=permutation_importance(sknnmodel, X, y, n_repeats=3,random_state=0)

In [15]:
#for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#    print(f"{X.columns[i]:<17}"
#        f"{r.importances_mean[i]:.3f}"
#        f" +/- {r.importances_std[i]:.3f}")

In [73]:
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd=pd.concat(results_pd, axis=1)

In [75]:
all_res_pd.iloc[:40]

,rank nn_auc recall1,perm. imp. nn_auc recall1,rank nn_auc recall0,perm. imp. nn_auc recall0
0,res_max,0.230714,bin_corine_28,0.055943
1,bin_corine_28,0.212473,bin_corine_26,0.042712
2,f81,0.205984,res_max,0.034084
3,ypos,0.141384,bin_corine_29,0.027792
4,bin_corine_26,0.122567,ypos,0.024232
5,xpos,0.111319,xpos,0.023046
6,bin_corine_29,0.110743,bin_corine_21,0.020673
7,dem,0.106849,min_dew_temp,0.019810
8,dom_vel,0.106417,f81,0.019127
9,bin_corine_21,0.087311,bin_corine_33,0.018264


In [17]:
all_res_pd.to_csv('results/permimpall.csv', index=False)